In [1]:
s = "My name is Kritik Mathur is Kritik".split()

In [11]:
k = "Kritik".split()

In [17]:
import os
from collections import defaultdict
import gensim
from sklearn import naive_bayes as nb
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier as mlpc
from sklearn.neighbors import KNeighborsClassifier
from numpy import mean

# suffixes = {
#     1: ["ो", "े", "ू", "ु", "ी", "ि", "ा"],
#     2: ["कर", "ाओ", "िए", "ाई", "ाए", "ने", "नी", "ना", "ते", "ीं", "ती", "ता", "ाँ", "ां", "ों", "ें"],
#     3: ["ाकर", "ाइए", "ाईं", "ाया", "ेगी", "ेगा", "ोगी", "ोगे", "ाने", "ाना", "ाते", "ाती", "ाता", "तीं", "ाओं", "ाएं", "ुओं", "ुएं", "ुआं"],
#     4: ["ाएगी", "ाएगा", "ाओगी", "ाओगे", "एंगी", "ेंगी", "एंगे", "ेंगे", "ूंगी", "ूंगा", "ातीं", "नाओं", "नाएं", "ताओं", "ताएं", "ियाँ", "ियों", "ियां"],
#     5: ["ाएंगी", "ाएंगे", "ाऊंगी", "ाऊंगा", "ाइयाँ", "ाइयों", "ाइयां"],
# }
#
# def stem(word):
#     for L in 5, 4, 3, 2, 1:
#         if len(word) > L + 1:
#             for suf in suffixes[L]:
#                 if word.endswith(suf):
#                     return word[:-L]
#     return word


stopwords = []
project_dir = "/home/rkb/Documents/IASNLP/"	#path where the project files are
stopwords_txt = project_dir + "stopwords"

# data_dir = "cleaned/"
# classes_dir = ["pos/", "neg/"]
# classes = ["pos", "neg"]
data_dir = "four_class_data/"				#path where the dataset is
classes_dir = ["E/", "A/", "C/", "D/"]
classes = ["E", "A", "C", "D"]

target = []
songs = []

def getCorpus():
    with open(stopwords_txt) as stops:
        for stop in stops:
            stopwords.append(stop.strip())

    for dir in classes_dir:
        folder = project_dir + data_dir + dir
        for file in os.listdir(folder):
            with open(folder + file) as f:
                song = f.read().strip().split()
                song = [word for word in song if word not in stopwords]
                songs.append(song)
                target.append(classes[classes_dir.index(dir)])

getCorpus()
frequency = defaultdict(int)
for song in songs:
    for word in song:
        frequency[word] += 1

songs = [[word for word in song if frequency[word] > 1] for song in songs]

dictionary = gensim.corpora.Dictionary(songs)
n_unique_tokens = len(dictionary)

bag_of_words = [dictionary.doc2bow(song) for song in songs]

#saving for persistency, can be avoided
gensim.corpora.MmCorpus.serialize('bag_of_words.mm', bag_of_words)
dictionary.save('dictionary.dict')

tfidf = gensim.models.TfidfModel(bag_of_words)
records = tfidf[bag_of_words]
dense_tfidf = gensim.matutils.corpus2dense(records, num_terms = n_unique_tokens).transpose()
dense_bow = gensim.matutils.corpus2dense(bag_of_words, num_terms = n_unique_tokens).transpose

kf = KFold(n_splits = 10, shuffle = True)

accuracies = []

for it in range(1):
    print ("Iteration ", it)
    for train, test in kf.split(dense_tfidf):
        train_set = []
        train_labels = []
        test_set = []
        test_labels = []
        for i in train:
            train_set.append(dense_tfidf[i])
            train_labels.append(target[i])
        for i in test:
            test_set.append(dense_tfidf[i])
            test_labels.append(target[i])
        print ("ANdar")

        # gnb_tfidf = nb.GaussianNB()
        # predicted = gnb_tfidf.fit(train_set, train_labels).predict(test_set)

        # gnb_bow = nb.GaussianNB()
        # predicted = gnb_bow.fit(train_set, train_labels).predict(test_set)

        # mnb_tfidf = svm.SVC()
        # predicted = mnb_tfidf.fit(train_set, train_labels).predict(test_set)

        knn_tfidf = KNeighborsClassifier()
        predicted = knn_tfidf.fit(train_set, train_labels).predict(test_set)

#         mlp_tfidf = mlpc(solver = 'lbfgs', hidden_layer_sizes = (15, 5), max_iter = 500)
#         predicted = mlp_tfidf.fit(train_set, train_labels).predict(test_set)

        #
        # for i in range (len(predicted)):
        #     print (predicted[i], " ", test_labels[i])
        # print ("****************************")

        # bern_tfidf = nb.

        print ((test_labels != predicted).sum())
        incorrect = (test_labels != predicted).sum()
        
        # incorrects.append(incorrect)
        accuracy = (len(test_set) - incorrect) / len(test_set) * 100.
        accuracies.append(accuracy)
        # print ("Number of mislabeled points out of a total %d points : %d" %(len(test_set), incorrect))
        print ("Accuracy : ", accuracy)

print ("Maximum accuracy attained ", max(accuracies))
print ("Average accuracy attained ", mean(accuracies))
# print ("Average accuracy %d" %((float(len(test_set)) - mean(incorrects)) / len(test_set)))


Iteration  0
ANdar
54
Accuracy :  47.572815534
ANdar
65
Accuracy :  36.8932038835
ANdar
53
Accuracy :  48.5436893204
ANdar
65
Accuracy :  36.8932038835
ANdar
60
Accuracy :  41.7475728155
ANdar
69
Accuracy :  33.0097087379
ANdar
61
Accuracy :  40.7766990291
ANdar
61
Accuracy :  40.7766990291
ANdar
62
Accuracy :  39.8058252427
ANdar
59
Accuracy :  42.1568627451
Maximum accuracy attained  48.5436893204
Average accuracy attained  40.8176280221


In [18]:
a = "My name is is kritik mathur is kritik".split()
k = "kritik"
l = a.index(k)

In [19]:
l

4

In [21]:
for i, j in enumerate(a):
    if j == k:
        print (i)

4
7


5 / 6

In [22]:
6 / 10

0.6